# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010. He drove for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [5]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 championship was won by Fernando Alonso, driving for Renault.


In [6]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 World Championship was won by Lewis Hamilton, driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [7]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [8]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [9]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
# Version 1

context_user = [
    {'role':'system', 'content':
     """You are a Sales expert and your task is to recommend customers products related to what they have bought recently for cross-selling.

     I've recently bought an iPhone 14 Pro Max.
     Product recommendation: I suggest you buy an iPhone 14 Pro Max case to protect your new phone.	

     I recently bought a new set of golf clubs.
     Product recommendation: I suggest you buy a golf bag to carry your new clubs and a set of golf balls to complete your golfing experience.

     I just bought a new house near the beach.
     Product recommendation: I suggest you buy a beach umbrella and a set of beach towels to enjoy your new home by the sea.
     """}
]
print(return_OAIResponse("I've recently bought a plane ticket to the Maldives.", context_user))

Product recommendation: I suggest you buy a good quality set of snorkeling gear to fully enjoy the crystal-clear waters of the Maldives. Additionally, consider purchasing a waterproof phone case to capture all the memorable moments during your trip.


In [12]:
# Version 2

context_user = [
    {'role':'system', 'content':
     """You are an avid reader who has a passion for books and literature and your task is to recommend books based on the topic about which the user is willing to read.

     I would like to read a book about what awaits for us in the future. 
     Book recommendation: I suggest you read "Sapiens: A Brief History of Humankind" by Yuval Noah Harari, which explores the history and future of our species.

     I am interested in learning more about the history of the Roman Empire.
     Book recommendation: I suggest you read "SPQR: A History of Ancient Rome" by Mary Beard, which provides a fascinating insight into the Roman Empire's history and culture.

     I want to read a book about the mysteries of the universe.
     Book recommendation: I suggest you read "Astrophysics for People in a Hurry" by Neil deGrasse Tyson, which offers a concise and engaging overview of the universe's most intriguing concepts.
     """}
]
print(return_OAIResponse("I am looking to read a book about the mysteries in the oceans.", context_user))

Book recommendation: I recommend you read "The Deep: The Extraordinary Creatures of the Abyss" by Claire Nouvian. This book delves into the mysteries of the deep sea with captivating imagery and fascinating insights into the mysterious world beneath the ocean's surface.


In [13]:
# Version 3

context_user = [
    {'role':'system', 'content':
     """You are an expert in managing social anxiety and your task is to recommend tricks based on the situations presented by the user on how to better approach other people and show confidence.

     I have a public presentation coming up and I'm feeling anxious about it.
     Trick recommendation: I suggest you practice deep breathing exercises before your presentation to calm your nerves and visualize yourself speaking confidently.

     I normally struggle when going to parties and meeting new people.
     Trick recommendation: I suggest you prepare a few conversation starters in advance, such as asking about the other person's interests or recent experiences, to help break the ice.

     I feel really uncomfortable when I am with someone else alone and I don't know what to say.
     Trick recommendation: I suggest you focus on active listening and ask open-ended questions to keep the conversation flowing, which can help ease your discomfort.
     """}
]
print(return_OAIResponse("I am 26 but I look like 18 years old. This is causing me some trouble when making friends, as they see me as a kid.", context_user))

Trick recommendation: I suggest you embrace your youthful appearance as a unique trait and use it as a conversation starter. You could make light-hearted jokes about it or share interesting experiences related to looking younger than your age to help others see you in a different light. Remember, confidence and authenticity go a long way in making connections with others.


 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong


In this case, the outputs were quite accurate and there weren't hallucinations. Probably because we were providing multiple shots, for the model to take as an example, which made the outputs much smoother and similar to what we requested. 